# Customer.IO Webhook Processing

This notebook demonstrates how to process Customer.IO webhooks for delivery tracking and event handling.

## Setup

First, let's import the webhook processing utilities.

In [ ]:
# Import webhook processing modules
from src.webhooks import (
    verify_signature,
    parse_event,
    get_event_type
)
import json

print("Customer.IO Webhook processing utilities imported successfully")

## Webhook Signature Verification

Verify that webhooks are authentically from Customer.IO.

In [ ]:
# Example webhook payload and signature
webhook_payload = '{"event_type": "email_delivered", "customer_id": "cust_123"}'
webhook_signature = "sha256=example_signature_here"
webhook_secret = "your_webhook_secret_here"

# Verify the signature
is_valid = verify_signature(
    payload=webhook_payload,
    signature=webhook_signature,
    secret=webhook_secret
)

print(f"Webhook signature valid: {is_valid}")

## Event Parsing

Parse webhook events and extract event types.

In [ ]:
# Parse the webhook event
event_data = parse_event(webhook_payload)
print(f"Parsed event data: {event_data}")

# Get the event type
event_type = get_event_type(event_data)
print(f"Event type: {event_type}")

## Complete Webhook Processing Workflow

A complete example of processing an incoming webhook.

In [ ]:
def process_webhook(payload, signature, secret):
    """Process a Customer.IO webhook with full validation."""
    
    # Step 1: Verify signature
    if not verify_signature(payload, signature, secret):
        print("❌ Invalid webhook signature")
        return None
    
    print("✅ Webhook signature verified")
    
    # Step 2: Parse event
    try:
        event = parse_event(payload)
        event_type = get_event_type(event)
        
        print(f"📧 Processing {event_type} event")
        
        # Step 3: Handle different event types
        if event_type == "email_delivered":
            print(f"✉️ Email delivered to customer {event.get('customer_id')}")
        elif event_type == "email_opened":
            print(f"👀 Email opened by customer {event.get('customer_id')}")
        elif event_type == "email_clicked":
            print(f"🖱️ Email link clicked by customer {event.get('customer_id')}")
        else:
            print(f"📝 Received {event_type} event")
        
        return event
        
    except Exception as e:
        print(f"❌ Error processing webhook: {e}")
        return None

# Example usage
result = process_webhook(webhook_payload, webhook_signature, webhook_secret)
print(f"Processing result: {result}")